In [1]:
%%javascript

window.load_remote_theme = true
var theme_js = "https://odhk.github.io/hyrule_theme/custom.js";

window.load_local_theme = function(){
    var hostname = document.location.hostname
    return ((hostname == "localhost" || hostname == '127.0.0.1') && !load_remote_theme)
}

var url = load_local_theme() ? document.location.origin + "/files/theme/custom.js" : theme_js

$.getScript(url)

<IPython.core.display.Javascript object>

In [1]:
import numpy as np
import pandas as pd
import gc
import lightgbm as lgb
import keras
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
import random
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Lambda, Embedding,constraints,\
Dropout, Activation,GRU,Bidirectional,Subtract, Permute, TimeDistributed, Reshape
from keras.layers import Conv1D,Conv2D,MaxPooling2D,GlobalAveragePooling1D,GlobalMaxPooling1D, MaxPooling1D, Flatten
from keras.layers import CuDNNGRU, CuDNNLSTM, SpatialDropout1D,Layer, initializers, regularizers
from keras.layers.merge import concatenate, Concatenate, Average, Dot, Maximum, Multiply, Subtract
from keras.models import Model
from keras.optimizers import RMSprop,Adam
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from keras import backend as K
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
import gc
import tensorflow as tf
from keras.activations import softmax

from keras.utils import plot_model
from keras.layers import *
from keras.applications.imagenet_utils import _obtain_input_shape
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.special import erfinv

import gensim
from gensim.models import word2vec
from gensim.models.word2vec import LineSentence
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

train = pd.read_csv('train_set.csv')
test = pd.read_csv('test_set.csv')

label = train['class']

char_embedding_index = {}
f = open('word_embedding_300dim_new2.txt')
for line in tqdm(f):
    values = line.split()
    word = values[0].lower()
    embeddings = np.asarray(values[1:], dtype = 'float32')
    char_embedding_index[word] = embeddings
f.close()

len(char_embedding_index)

api_embedding_file_path = 'word_embedding_300dim_new2.txt'
embedding_dim = 300
max_nb_api = len(char_embedding_index) + 1
max_sequence_length_api =1000

all_word = list(train['word_seg'].values) + list(test['word_seg'].values)

all_word_list = []
for i in list(all_word):
    all_word_list.append(i.split(' '))

len(all_word_list)

train['word_seg'].apply(lambda x:len(x.split(' '))).describe()

data = pd.DataFrame()

gc.collect()

tokenizer_api = Tokenizer()

tokenizer_api.fit_on_texts(all_word_list)

api_seq = tokenizer_api.texts_to_sequences(all_word_list)

data['word_seq'] = pad_sequences(api_seq, maxlen = max_sequence_length_api, padding='post').tolist()

api_index = tokenizer_api.word_index
print('Found %s unique tokens' % len(api_index))
nb_words = min(max_nb_api, len(char_embedding_index)+1)
api_embedding_matrix = np.zeros((nb_words, embedding_dim))

for word, i in tqdm(api_index.items()):
    embedding_vector = char_embedding_index.get(word)
    if embedding_vector is not None:
        api_embedding_matrix[i] = embedding_vector
    else:
        print('no char%s'%word)

data['id'] = train['id']

train_FE = data.iloc[:len(label),:]
test_FE = data.iloc[len(label):,:]

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

def get_input(df):
    
    _input = [np.array(df.word_seq.values.tolist())]
    
    
    return _input 

/home/libo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
1271461it [01:41, 12520.13it/s]
  2%|▏         | 31325/1271460 [00:00<00:03, 313180.86it/s]

Found 1271460 unique tokens


100%|██████████| 1271460/1271460 [00:04<00:00, 303195.73it/s]


In [2]:
def attention(x):
    
    q1, q2 = x[0], x[1]
    
    #compute match matrix euclidean match score
    match_matrix = 1. + K.sqrt(
        -2 * K.batch_dot(q1, q2, axes=[2, 2]) +
        K.expand_dims(K.sum(K.square(q1), axis=2), 2) +
        K.expand_dims(K.sum(K.square(q2), axis=2), 1)
    )
    
    match_matrix = K.maximum(match_matrix, K.epsilon())
    match_matrix = 1. / match_matrix
    #compute attention output
    q1_new = K.batch_dot(K.softmax(match_matrix, axis = 1), q2)
    match_matrix_T = Permute((2, 1))(match_matrix)
    q2_new = K.batch_dot(K.softmax(match_matrix_T, axis = 1), q1)
    
    return [q1_new, q2_new]

In [9]:
def Globel_attention_DR_Stack_RNN():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    #RNN
    char_rnn_layer_1 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    char_rnn_layer_2 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    
    char_embedding = api_embedding_layer(char_input)
    char_embedding = SpatialDropout1D(0.2)(char_embedding)
    
    #双层BiLSTM
    char_rnn_1= char_rnn_layer_1(char_embedding)
    char_rnn_2= char_rnn_layer_2(char_rnn_1)
    
    #max avg 
    char_att_maxpooling_1 = GlobalMaxPooling1D()(char_rnn_1)
    char_att_avgpooling_1 = GlobalAveragePooling1D()(char_rnn_1)
    
    char_att_maxpooling_2 = GlobalMaxPooling1D()(char_rnn_2)
    char_att_avgpooling_2 = GlobalAveragePooling1D()(char_rnn_2)
    
    #batchnormalization
    char_embedding = BatchNormalization()(char_embedding)
    char_rnn_2 = BatchNormalization()(char_rnn_2)
    
    #globel attention information
    r = Lambda(attention, name = 'attention')([char_embedding, char_rnn_2])
    
    attention_map_left = r[0]
    attention_map_right = r[1]
    
    att_maxpooling_left = GlobalMaxPooling1D()(attention_map_left)
    att_maxpooling_right = GlobalMaxPooling1D()(attention_map_right)
    
    att_avgpooling_left = GlobalAveragePooling1D()(attention_map_left)
    att_avgpooling_right = GlobalAveragePooling1D()(attention_map_right)
   
    max_sub_char_abs_attention = Lambda(lambda x:K.abs(x[0] - x[1]))([att_maxpooling_left, att_maxpooling_right])
    max_multi_char_attention = Lambda(lambda x: x[0] * x[1])([att_maxpooling_left, att_maxpooling_right])

    avg_sub_char_abs_attention = Lambda(lambda x:K.abs(x[0] - x[1]))([att_avgpooling_left, att_avgpooling_right])
    avg_multi_char_attention = Lambda(lambda x: x[0] * x[1])([att_avgpooling_left, att_avgpooling_right])

    merged = concatenate([char_att_maxpooling_1, char_att_avgpooling_1, \
                          char_att_maxpooling_2, char_att_avgpooling_2,\
                         max_sub_char_abs_attention, max_multi_char_attention, \
                         avg_sub_char_abs_attention, avg_multi_char_attention, \
                         att_maxpooling_left, att_maxpooling_right, \
                         att_avgpooling_left, att_avgpooling_right])
    merged = Dropout(0.4)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(512, activation='relu')(merged)
    merged = Dropout(0.1)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [4]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [5]:
del train, test, all_word
gc.collect()

48

In [ ]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64

    model = Globel_attention_DR_Stack_RNN()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=13, batch_size=BATCH_SIZE, shuffle=True)#callbacks=callbacks

    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/13
92040/92040 [==============================] - 568s 6ms/step - loss: 1.1911 - acc: 0.6562 - val_loss: 0.8888 - val_acc: 0.7348
Epoch 2/13
92040/92040 [==============================] - 557s 6ms/step - loss: 0.9251 - acc: 0.7281 - val_loss: 0.8385 - val_acc: 0.7514
Epoch 3/13
92040/92040 [==============================] - 556s 6ms/step - loss: 0.8498 - acc: 0.7488 - val_loss: 0.8050 - val_acc: 0.7594
Epoch 4/13
92040/92040 [==============================] - 556s 6ms/step - loss: 0.8060 - acc: 0.7592 - val_loss: 0.7763 - val_acc: 0.7716
Epoch 5/13
92040/92040 [==============================] - 555s 6ms/step - loss: 0.7754 - acc: 0.7671 - val_loss: 0.7492 - val_acc: 0.7755
Epoch 6/13
92040/92040 [==============================] - 554s 6ms/step - loss: 0.7402 - acc: 0.7787 - val_loss: 0.7490 - val_acc: 0.7760
Epoch 7/13
92040/92040 [==============================] - 555s 6ms/step - loss: 0.7145 - acc: 0.7832 - val

Epoch 8/13
92049/92049 [==============================] - 576s 6ms/step - loss: 0.6888 - acc: 0.7905 - val_loss: 0.7249 - val_acc: 0.7825
Epoch 9/13
92049/92049 [==============================] - 577s 6ms/step - loss: 0.6635 - acc: 0.7974 - val_loss: 0.7248 - val_acc: 0.7806
Epoch 10/13
92049/92049 [==============================] - 578s 6ms/step - loss: 0.6398 - acc: 0.8029 - val_loss: 0.7102 - val_acc: 0.7852
Epoch 11/13
92049/92049 [==============================] - 578s 6ms/step - loss: 0.6185 - acc: 0.8085 - val_loss: 0.7180 - val_acc: 0.7855
Epoch 12/13
92049/92049 [==============================] - 578s 6ms/step - loss: 0.5952 - acc: 0.8157 - val_loss: 0.7168 - val_acc: 0.7852
Epoch 13/13
102277/102277 [==============================] - 182s 2ms/step
*******5*******
Train on 92051 samples, validate on 10226 samples
Epoch 1/13
92051/92051 [==============================] - 588s 6ms/step - loss: 1.1887 - acc: 0.6564 - val_loss: 0.9422 - val_acc: 0.7224
Epoch 2/13
92051/92051 [====

Epoch 10/13
92056/92056 [==============================] - 578s 6ms/step - loss: 0.6475 - acc: 0.8019 - val_loss: 0.6985 - val_acc: 0.7882
Epoch 11/13
92056/92056 [==============================] - 577s 6ms/step - loss: 0.6240 - acc: 0.8083 - val_loss: 0.7067 - val_acc: 0.7862
Epoch 12/13
92056/92056 [==============================] - 577s 6ms/step - loss: 0.6043 - acc: 0.8136 - val_loss: 0.6977 - val_acc: 0.7931
Epoch 13/13
102277/102277 [==============================] - 186s 2ms/step
*******9*******
Train on 92057 samples, validate on 10220 samples
Epoch 1/13
92057/92057 [==============================] - 586s 6ms/step - loss: 1.2205 - acc: 0.6486 - val_loss: 1.1783 - val_acc: 0.6704
Epoch 2/13
92057/92057 [==============================] - 576s 6ms/step - loss: 0.9399 - acc: 0.7238 - val_loss: 0.8672 - val_acc: 0.7455
Epoch 3/13
92057/92057 [==============================] - 577s 6ms/step - loss: 0.8608 - acc: 0.7457 - val_loss: 0.7979 - val_acc: 0.7601
Epoch 4/13
92057/92057 [====

In [23]:
1

1

In [11]:
meta_train.columns = ['meta_rnn_att_wrod_v2_dim_' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [12]:
meta_train.to_csv('/home/libo/daguan/stack/train_meta_rnn_att_wrod_v2_300dim.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack/test_meta_rnn_att_wrod_v2_300dim.csv', index = None ,encoding = 'utf-8')

In [ ]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64
    bst_model_path = '10folds_' + 'RNN_attention_char_v1_300dim' + str(i) + '.hdf5'
    early_stopping =EarlyStopping(monitor='val_acc', patience=2)
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    callbacks = [
            early_stopping,
            model_checkpoint
        ]
    model = Globel_attention_DR_Stack_RNN()
    model.compile(optimizer='adam', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=20, batch_size=BATCH_SIZE, shuffle=True, callbacks=callbacks)#callbacks=callbacks
    
    model = Globel_attention_DR_Stack_RNN()
    model.load_weights(bst_model_path)
    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/20
92040/92040 [==============================] - 575s 6ms/step - loss: 1.1914 - acc: 0.6556 - val_loss: 0.9431 - val_acc: 0.7201
Epoch 2/20
92040/92040 [==============================] - 559s 6ms/step - loss: 0.9247 - acc: 0.7285 - val_loss: 0.8576 - val_acc: 0.7453
Epoch 3/20
92040/92040 [==============================] - 555s 6ms/step - loss: 0.8525 - acc: 0.7476 - val_loss: 0.8103 - val_acc: 0.7564
Epoch 4/20
92040/92040 [==============================] - 556s 6ms/step - loss: 0.8074 - acc: 0.7589 - val_loss: 0.7801 - val_acc: 0.7655
Epoch 5/20
92040/92040 [==============================] - 556s 6ms/step - loss: 0.7756 - acc: 0.7690 - val_loss: 0.7749 - val_acc: 0.7655
Epoch 6/20
92040/92040 [==============================] - 553s 6ms/step - loss: 0.7442 - acc: 0.7767 - val_loss: 0.7687 - val_acc: 0.7701
Epoch 7/20
92040/92040 [==============================] - 554s 6ms/step - loss: 0.7147 - acc: 0.7834 - val

92056/92056 [==============================] - 581s 6ms/step - loss: 0.7445 - acc: 0.7770 - val_loss: 0.7268 - val_acc: 0.7822
Epoch 7/20
92056/92056 [==============================] - 581s 6ms/step - loss: 0.7184 - acc: 0.7832 - val_loss: 0.7106 - val_acc: 0.7833
Epoch 8/20
92056/92056 [==============================] - 581s 6ms/step - loss: 0.6884 - acc: 0.7916 - val_loss: 0.7058 - val_acc: 0.7887
Epoch 9/20
92056/92056 [==============================] - 580s 6ms/step - loss: 0.6646 - acc: 0.7971 - val_loss: 0.6965 - val_acc: 0.7882
Epoch 10/20
92056/92056 [==============================] - 580s 6ms/step - loss: 0.6448 - acc: 0.8025 - val_loss: 0.6926 - val_acc: 0.7921
Epoch 11/20
92056/92056 [==============================] - 580s 6ms/step - loss: 0.6192 - acc: 0.8097 - val_loss: 0.6952 - val_acc: 0.7909
Epoch 12/20
102277/102277 [==============================] - 185s 2ms/step
*******9*******
Train on 92057 samples, validate on 10220 samples
Epoch 1/20
92057/92057 [================

In [10]:
meta_train.shape

(102277, 19)

In [11]:
meta_test.shape

(102277, 190)

In [12]:
meta_train.columns = ['meta_base_rnn_att_wrod_v1_300dim_' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [13]:
p.shape, meta_train.shape

((102277, 19), (102277, 19))

In [14]:
meta_train.to_csv('/home/libo/daguan/stack_new/train_meta_base_att_rnn_word_v2_300dim.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack_new/test_meta_base_att_rnn_word_v2_300dim.csv', index = None ,encoding = 'utf-8')

In [15]:
test = pd.read_csv('test_set.csv')

pred = pd.DataFrame(test['id'])

pred['class'] = np.argmax(p.values, axis=1) + 1

pred.to_csv('/home/libo/daguan/stack_new/9.4.2.csv', index = None ,encoding = 'utf-8')

In [20]:
def Globel_attention_DR_Stack_RNN():
    
    #Input layer
    char_input = Input(shape=(max_sequence_length_api,), dtype='int32')
    
    api_embedding_layer = Embedding(nb_words,
        embedding_dim,
        weights=[api_embedding_matrix],
        input_length=max_sequence_length_api,
        trainable=False)
    
    #RNN
    char_rnn_layer_1 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    char_rnn_layer_2 = Bidirectional(CuDNNLSTM(150, 
                               return_sequences=True))
    
    char_embedding = api_embedding_layer(char_input)
    #char_embedding = SpatialDropout1D(0.2)(char_embedding)
    
    #双层BiLSTM
    char_rnn_1= char_rnn_layer_1(char_embedding)
    char_rnn_2= char_rnn_layer_2(char_rnn_1)
    
    #max avg 
    char_att_maxpooling_1 = GlobalMaxPooling1D()(char_rnn_1)
    char_att_avgpooling_1 = GlobalAveragePooling1D()(char_rnn_1)
    
    char_att_maxpooling_2 = GlobalMaxPooling1D()(char_rnn_2)
    char_att_avgpooling_2 = GlobalAveragePooling1D()(char_rnn_2)
    
    #batchnormalization
    char_embedding = BatchNormalization()(char_embedding)
    char_rnn_2 = BatchNormalization()(char_rnn_2)
    
    #globel attention information
    r = Lambda(attention, name = 'attention')([char_embedding, char_rnn_2])
    
    attention_map_left = r[0]
    attention_map_right = r[1]
    
    att_maxpooling_left = GlobalMaxPooling1D()(attention_map_left)
    att_maxpooling_right = GlobalMaxPooling1D()(attention_map_right)
    
    att_avgpooling_left = GlobalAveragePooling1D()(attention_map_left)
    att_avgpooling_right = GlobalAveragePooling1D()(attention_map_right)
   
    max_sub_char_abs_attention = Lambda(lambda x:K.abs(x[0] - x[1]))([att_maxpooling_left, att_maxpooling_right])
    max_multi_char_attention = Lambda(lambda x: x[0] * x[1])([att_maxpooling_left, att_maxpooling_right])

    avg_sub_char_abs_attention = Lambda(lambda x:K.abs(x[0] - x[1]))([att_avgpooling_left, att_avgpooling_right])
    avg_multi_char_attention = Lambda(lambda x: x[0] * x[1])([att_avgpooling_left, att_avgpooling_right])

    merged = concatenate([char_att_maxpooling_1, char_att_avgpooling_1, \
                          char_att_maxpooling_2, char_att_avgpooling_2,\
                         max_sub_char_abs_attention, max_multi_char_attention, \
                         avg_sub_char_abs_attention, avg_multi_char_attention, \
                         att_maxpooling_left, att_maxpooling_right, \
                         att_avgpooling_left, att_avgpooling_right])
    merged = Dropout(0.3)(merged)
    merged = BatchNormalization()(merged)
    merged = Dense(400, activation='relu')(merged)
    merged = Dropout(0.2)(merged)
    merged = BatchNormalization()(merged) 
    preds = Dense(19, activation = 'softmax')(merged)
    
    model = Model(inputs=[char_input],outputs=preds)
    return model

In [21]:
meta_train = pd.DataFrame(np.array([None]*train_FE.shape[0]*19).reshape(train_FE.shape[0], 19))
meta_train.columns = ['prob' + str(x) for x in range(19)]
meta_test = pd.DataFrame()

label = train['class']

label = label - 1

categorical_labels = keras.utils.np_utils.to_categorical(label, num_classes=19)

index = pd.read_csv('fold.csv')
index['id'] = train['id']

In [ ]:
%%time
#logloss_list = []
for i in range(10):
    gc.collect()
    K.clear_session()
    print('*******%s*******'%i)
    idx_train = np.array(index[index['fold'] != i]['id'])
    idx_val = np.array(index[index['fold'] == i]['id'])
    
    
    X_train = train_FE.loc[idx_train, :]
    y_train = categorical_labels[idx_train]

    X_val = train_FE.loc[idx_val, :]
    y_val = categorical_labels[idx_val]

    #dtest = test.loc[:,user_col]
    
    X_train = get_input(X_train)
    X_val = get_input(X_val)
    X_test = get_input(test_FE)
    
    BATCH_SIZE = 64

    model = Globel_attention_DR_Stack_RNN()
    model.compile(optimizer='RMSprop', loss='categorical_crossentropy',\
                 metrics=['accuracy'])
    

    hist = model.fit(X_train, y_train, \
            validation_data=(X_val, y_val), \
            epochs=13, batch_size=BATCH_SIZE, shuffle=True)#callbacks=callbacks

    
    pred = pd.DataFrame(model.predict(X_val,batch_size=128,verbose=1))
    
    for i in range(19):
        meta_train.loc[idx_val, 'prob' + str(i)] = pred[i].values
    
    prob = model.predict(X_test,batch_size=128,verbose=1)
    meta_test = pd.concat([meta_test, pd.DataFrame(prob)], axis = 1)

*******0*******
Train on 92040 samples, validate on 10237 samples
Epoch 1/13
92040/92040 [==============================] - 558s 6ms/step - loss: 1.0630 - acc: 0.6951 - val_loss: 1.0005 - val_acc: 0.7154
Epoch 2/13
92040/92040 [==============================] - 557s 6ms/step - loss: 0.8400 - acc: 0.7532 - val_loss: 0.8633 - val_acc: 0.7480
Epoch 3/13
92040/92040 [==============================] - 557s 6ms/step - loss: 0.7764 - acc: 0.7708 - val_loss: 0.9347 - val_acc: 0.7425
Epoch 4/13
92040/92040 [==============================] - 556s 6ms/step - loss: 0.7270 - acc: 0.7835 - val_loss: 0.7821 - val_acc: 0.7723
Epoch 5/13
92040/92040 [==============================] - 556s 6ms/step - loss: 0.6890 - acc: 0.7942 - val_loss: 0.8370 - val_acc: 0.7575
Epoch 6/13
92040/92040 [==============================] - 558s 6ms/step - loss: 0.6499 - acc: 0.8043 - val_loss: 0.7719 - val_acc: 0.7740
Epoch 7/13
92040/92040 [==============================] - 556s 6ms/step - loss: 0.6127 - acc: 0.8151 - val

In [24]:
meta_train.shape, meta_test.shape

((102277, 19), (102277, 190))

In [25]:
meta_train.columns = ['meta_base_rnn_att_wrod_v4_300dim_' + x for x in meta_train.columns]

p = pd.DataFrame()

for i in range(19):
    p[i] = meta_test[i].mean(axis = 1)

p.columns = meta_train.columns

In [26]:
meta_train.to_csv('/home/libo/daguan/stack_new/train_meta_base_att_rnn_word_v4_300dim.csv', index = None ,encoding = 'utf-8')

p.to_csv('/home/libo/daguan/stack_new/test_meta_base_att_rnn_word_v4_300dim.csv', index = None ,encoding = 'utf-8')

In [ ]:
1